In [0]:
%run ./azurehelper/azurehelper

In [0]:
dbutils.library.restartPython()

# user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
# print(user)

In [0]:
import shutil
import os
import glob

from mlflow.tracking import MlflowClient


def export_model_to_storage(model_name: str, model_version: str, azure_helper: AzureHelper):

    print('Getting storage credentials ...')

    blob_name = azure_helper.get_blob_name(model_name, model_version)

    client = MlflowClient(registry_uri="databricks")

    # --- DOWNLOAD MODEL FROM REGISTRY ---
    model_uri = f"models:/{model_name}/{model_version}"
    local_dir = 'model'
    os.makedirs(local_dir, exist_ok=True)
    mv = client.get_model_version(name=model_name, version=model_version)

    print(f"Downloading model from {model_uri}...")
    local_path = client.download_artifacts(run_id=mv.run_id, path=local_dir)

    artifact = f'{local_path}/artifacts'
    zip_path = f'{artifact}/{model_name}_{model_version}.zip'

    # print("Listing files in", artifact)
    # for file in glob.glob(f"{artifact}/**", recursive=True):
    #     print(file)

    print('zip_path', zip_path)

    print('Uploading to Azure Storage')

    target_blob_file = azure_helper.get_target_blob_file(blob_name)
    print(f"Uploading to {target_blob_file}...")

    dbutils.fs.cp(f"file:{zip_path}", target_blob_file)

    print('UPLOADED')

    return blob_name

azure_helper = AzureHelper()

model_name = "aisdk_model_state_identifier-edge"
model_version = 1

blob_name = export_model_to_storage(model_name, model_version, azure_helper)
    


In [0]:


azure_helper.generate_sas_token(blob_name)